# Springboard--DSC Program

# Capstone Project 1 - Data Wrangling 
### by Ellen A. Savoye

The data collected is from a Kaggle competition, Jigsaw Unintended Bias in Toxicity Classification, via the Kaggle API. Of the 7 files in the zipped data, we will be focusing on the 'train' data. The original 'train' data is comprised of 45 columns containing information on toxicity and identity labels, comments, and metadata. 

### Import packages and data

In [3]:
# !pip install kaggle
# !pip install spacy
# !pip install spacymoji
# !pip install emot
# !pip install demoji
# !pip install swifter

In [2]:
import pandas as pd
import numpy as np

# libraries for NLP
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata
import string
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import demoji
demoji.download_codes()
import swifter

# libraries for getting and moving data
import os
from os import path
import shutil
from zipfile import ZipFile

from kaggle.api.kaggle_api_extended import KaggleApi

... OK (Got response in 0.38 seconds)
Writing emoji data to C:\Users\ellen\.demoji/codes.json ...
... OK


In [4]:
# necessary dependencies for text pre-processing

nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [5]:
# Set directories
src = "C:\\Users\\ellen\\Documents\\GitHub\\Data_Science_Career_Track\\Capstone_1\\Code\\"
dst = "C:\\Users\\ellen\\Documents\\GitHub\\Data_Science_Career_Track\\Capstone_1\\Data\\"

kaggle_comp_name = 'jigsaw-unintended-bias-in-toxicity-classification'
zipfile_name = kaggle_comp_name + '.zip'

csv_file = [i for i in os.listdir(dst) if i.startswith("train") and path.isfile(path.join(dst, i))]
zip_file = [i for i in os.listdir(dst) if i.startswith("jigsaw") and path.isfile(path.join(dst, i))]

if zip_file[0] != zipfile_name:
    #Import data from Kaggle API
    api = KaggleApi()
    api.authenticate()
    files = api.competition_download_files(kaggle_comp_name)
    
    # Move the jigsaw zip file to the Data folder
    files = [i for i in os.listdir(src) if i.startswith("jigsaw") and path.isfile(path.join(src, i))]
    for f in files:
        shutil.move(path.join(src, f), dst)
    
    # Check if Train data is already extracted
    if csv_file != 'train.csv':
        with ZipFile(dst + zipfile_name, 'r') as zipObj:
            # Extract all the contents of zip file in current directory
            print(zipObj.namelist())
            zipObj.extract('train.csv', path = dst)
else:
    print('Data is already downloaded')

Data is already downloaded


In [6]:
# Read in the train dataset
csv_filename = 'train.csv'
train_data = pd.read_csv(dst + csv_filename, low_memory=False)

# Output the number of rows
print("Total rows: {0}".format(len(train_data)))

# See which headers are available
print(list(train_data))

Total rows: 1804874
['id', 'target', 'comment_text', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual', 'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu', 'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability', 'jewish', 'latino', 'male', 'muslim', 'other_disability', 'other_gender', 'other_race_or_ethnicity', 'other_religion', 'other_sexual_orientation', 'physical_disability', 'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date', 'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree', 'sexual_explicit', 'identity_annotator_count', 'toxicity_annotator_count']


The metadata from Civil Comments platform is contained in the following columns: created_date, publication_id, parent_id, article_id, rating, funny, wow, sad, likes, and disagree.

I will be keeping these fields in for further exploration and possible use.

In [6]:
#function for counting null records:
def num_missing(x):
    return sum(x.isnull())

#Applying per column:
print("Missing values per column:")
print(train_data.apply(num_missing, axis=0))

Missing values per column:
id                                           0
target                                       0
comment_text                                 0
severe_toxicity                              0
obscene                                      0
identity_attack                              0
insult                                       0
threat                                       0
asian                                  1399744
atheist                                1399744
bisexual                               1399744
black                                  1399744
buddhist                               1399744
christian                              1399744
female                                 1399744
heterosexual                           1399744
hindu                                  1399744
homosexual_gay_or_lesbian              1399744
intellectual_or_learning_disability    1399744
jewish                                 1399744
latino                           

In [7]:
# Find percent of missing values for each column instead of number of records

percent_missing = train_data.isnull().sum() * 100 / len(train_data)
print(round(percent_missing,1))

id                                      0.0
target                                  0.0
comment_text                            0.0
severe_toxicity                         0.0
obscene                                 0.0
identity_attack                         0.0
insult                                  0.0
threat                                  0.0
asian                                  77.6
atheist                                77.6
bisexual                               77.6
black                                  77.6
buddhist                               77.6
christian                              77.6
female                                 77.6
heterosexual                           77.6
hindu                                  77.6
homosexual_gay_or_lesbian              77.6
intellectual_or_learning_disability    77.6
jewish                                 77.6
latino                                 77.6
male                                   77.6
muslim                          

In [8]:
# Data type of each column

train_data.dtypes

id                                       int64
target                                 float64
comment_text                            object
severe_toxicity                        float64
obscene                                float64
identity_attack                        float64
insult                                 float64
threat                                 float64
asian                                  float64
atheist                                float64
bisexual                               float64
black                                  float64
buddhist                               float64
christian                              float64
female                                 float64
heterosexual                           float64
hindu                                  float64
homosexual_gay_or_lesbian              float64
intellectual_or_learning_disability    float64
jewish                                 float64
latino                                 float64
male         

In [9]:
train_data.rating.unique()

array(['rejected', 'approved'], dtype=object)

In [10]:
train_data.shape

(1804874, 45)

There are 45 columns in the train dataframe. Of those 45, only three columns, 'comment_text', 'created_date', and 'rating', are objects. The remaining 42 are either float64 or int64. These columns are non-categorical. 'Comment_text' is categorical containing the individual comments that we need to analyze. 'Created_date' contains the original date the comments were created. 'Rating' is a categorical containing two values: rejected or approved. 

In [11]:
train_data.comment_text.head(5)

0    This is so cool. It's like, 'would you want yo...
1    Thank you!! This would make my life a lot less...
2    This is such an urgent design problem; kudos t...
3    Is this something I'll be able to install on m...
4                 haha you guys are a bunch of losers.
Name: comment_text, dtype: object

In [12]:
# View unique records in a particular column

#sorted(train_data.target.unique())
train_data.target.unique()

array([0.        , 0.89361702, 0.66666667, ..., 0.87726476, 0.01116838,
       0.87008821])

In [13]:
# Checking for blank string records
np.where(train_data.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [14]:
# Checking the range of numerical columns
print('Minimum value: ')
train_data.iloc[:,:].min()

Minimum value: 


id                                                                                 59848
target                                                                                 0
comment_text                           Canada is north of the USA border,  its colde...
severe_toxicity                                                                        0
obscene                                                                                0
identity_attack                                                                        0
insult                                                                                 0
threat                                                                                 0
asian                                                                                  0
atheist                                                                                0
bisexual                                                                               0
black                

In [15]:
print('Maximum value: ')
train_data.iloc[:,:].max()

Maximum value: 


id                                                           6334010
target                                                             1
comment_text                                         🤣gotta love it!
severe_toxicity                                                    1
obscene                                                            1
identity_attack                                                    1
insult                                                             1
threat                                                             1
asian                                                              1
atheist                                                            1
bisexual                                                           1
black                                                              1
buddhist                                                           1
christian                                                          1
female                            

Toxicity and identity labels range from 0.0-1.0. The value represents the fraction of raters who believed the label fit the comment. Toxicity labels do not have any missing values. According to the competition details, a subset of comments have been labeled with a variety of identity attributes that have been mentioned in the comment. As such, every identity label is missing ~78% of the values per column. The subset comprises approximately 22% of the data. 

Two examples of how labeling works are as follows:
Example 1: 
    - Comment: I'm a white woman in my late 60's and believe me, they are not too crazy about me either!!
    - Toxicity Labels: All 0.0
    - Identity Mention Labels: female: 1.0, white: 1.0 (all others 0.0)

Example 2: 
    - Comment: Continue to stand strong LGBT community. Yes, indeed, you'll overcome and you have.
    - Toxicity Labels: All 0.0
    - Identity Mention Labels: homosexual_gay_or_lesbian: 0.8, bisexual: 0.6, transgender: 0.3 (all others 0.0)


'Target' is the toxicity label. 'Severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', and 'sexual_explicit' are toxicity sub types. All toxicity labels can be converted to categorical variables by using >= 0.5 as a positive indicator (1). 

Aside from 'id', 'comment_text', 'identity_annotator_count' and 'toxicity_annotator_count', the same conversion can be applied to the remaining identity columns. 'Id' is a unique identifier for each comment but may not hold value to keep in the data frame. 'Identity_annotator_count' and 'toxicity_annotator_count' are metadata columns from Jigsaw and may not hold value either. However, I'm not removing them until I do my exploratory data analysis to determine if they offer valuable insights. 

# Text Pre-processing and Cleaning

Given the size of the original dataset, ~1.8M records, I took a 25% random sample to quantify and test some of the text cleaning and pre-processing ideas: testing for punctuation, emoticons, emoji, accented words, etc. 

In [7]:
# Creating a smaller 25% random sample data frame to test text cleaning/pre-processing

subset_train_data = train_data.sample(frac=0.25)

In [8]:
subset_train_data.shape

(451218, 45)

Prior to considering removing emojis and emoticons, I need to see how frequently they occur in my data.

In [9]:
%timeit
#extracting the emojis
emojis_list = map(lambda x: ''.join(x.split()), UNICODE_EMO.keys())
r = re.compile('|'.join(re.escape(p) for p in emojis_list))
#English
subset_train_data['emoji'] = subset_train_data['comment_text'].str.findall(r)
#Emoji count
subset_train_data['emoji_count'] = subset_train_data['emoji'].swifter.apply(len, axis=1)

KeyboardInterrupt: 

In [21]:
print('Emoji percentage in sample data: ', round(subset_train_data['emoji_count'].sum() / subset_train_data['emoji_count'].count() * 100,4), '%')

Emoji percentage in sample data:  0.5447 %


In [19]:
#extracting the emoticons
emoticons_list = map(lambda x: ''.join(x.split()), EMOTICONS.keys())
r = re.compile('|'.join(re.escape(p) for p in emoticons_list))
#English
subset_train_data['emoticons'] = subset_train_data['comment_text'].str.findall(r)
#Emoji count
subset_train_data['emoticons_count'] = subset_train_data['emoticons'].apply(len)

In [20]:
print('Emoticons percentage in sample data: ', round(subset_train_data['emoticons_count'].sum() / subset_train_data['emoticons_count'].count() * 100,4), '%')

Emoticons percentage in sample data:  4.1268 %


My sample data contains emoticons in approximately 4% of the data and emoji in .5%. Given the minimal presence of both emoji and emoticons, I'll be removing them both along with punctuation before moving on.

In [73]:
%timeit

subset_train_data.loc[:, 'comment_text_no_emo'] = subset_train_data.loc[:,'comment_text'].apply(lambda x: demoji.replace(x)) 

In [77]:
list(subset_train_data[subset_train_data['emoji_count'] == 1].comment_text.tail())

['Back in 2004, there was a movie titled "A Day Without a Mexican" satirically depicting how the economy of California would collapse without the very people whom the "respectable" people loved to despise. \'Nuff said 😎',
 "We're trying, but there she was back on TV this week throwing around wild excuses for why she lost again.🤓",
 'Lane Community College: Climbing \xadPoeTree.\nAn amazing poetry group\nMLK💜',
 "Mind you we just had a 15 and 17yr old rob a bank in the middle of a school day no less.  Have to wonder what the hell these parents are doing, because it isn't parenting.  \n\nIt isn't laying down the law and demanding kids respect their elders.  We have a bunch of kids running around acting complete criminal fools. Spanking is horrible and teaches violence? Talk to them as adults? Yeah as we can see that isn't working out to well now is it Dr Spock. \n\nAll of those involved are freaking kids. All of them involved knew what happened and then you get one who speaks up, why? Be

In [45]:
test = subset_train_data[subset_train_data['emoji_count'] == 1].comment_text.tail(1)

In [51]:
print(type(test))

<class 'pandas.core.series.Series'>


In [48]:
print(list(convert_emojis(test)))

['Go rockies! Big win although i think they should get rid of a couple of relievers like otto and dunn, send them down to the minors and bring up some fresh talent 😁']


# TBD: 
I'm going to split each comment into a list of words before removing punctuation to preserve the integrity of contracted words, i.e. we're or won't.

In [22]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [23]:
subset_train_data.loc[:,'comment_text_noaccent'] = subset_train_data.loc[:,'comment_text'].apply(lambda x: remove_accented_chars(x)) 

In [24]:
subset_train_data['compare_ifaccent'] = np.where(subset_train_data['comment_text_noaccent'] != subset_train_data['comment_text'], 1, 0)

In [25]:
print('Number of accent characters: ', round(subset_train_data['compare_ifaccent'].sum() / subset_train_data['compare_ifaccent'].count() * 100,4), '%')

Number of accent characters:  4.3478 %


In [35]:
print(list(subset_train_data[subset_train_data['compare_ifaccent'] == 1].comment_text_noaccent.head(2)))

["'\n...I just talked to God and he said I can ignore such craven egotistical nonsense.\n\n.\nAs if you're the spokesman for God.   Awesome, ...now that's an ego trip.\n.", '(Part two of two)  O God, let all the nations praise you! in their assemblies and in searching for you in truth, in truth not in the sense of verily, but in the sense of loving the results of research.  Leviticus 25:17) has it right, do not deal unfairly (with scientists), then; but stand in fear of your god [sic].  I, the LORD, am your God and the author of all truth, no matter what you do with new findings.  Liturgy of the Word, Reading 406, Saturday of the Seventeenth Week in Ordinary Time I.']


In [36]:
print(list(subset_train_data[subset_train_data['compare_ifaccent'] == 1].comment_text.head(2)))

["'\n…I just talked to God and he said I can ignore such craven egotistical nonsense.\n\n.\nAs if you're the spokesman for God.   Awesome, …now that's an ego trip.\n.", '(Part two of two)  “O God, let all the nations praise you!” in their assemblies and in searching for you in truth, in truth not in the sense of verily, but in the sense of loving the results of research.  Leviticus 25:17) has it right, “do not deal unfairly (with scientists), then; but stand in fear of your god [sic].  I, the LORD, am your God” and the author of all truth, no matter what you do with new findings.  Liturgy of the Word, Reading 406, Saturday of the Seventeenth Week in Ordinary Time I.']


what can be deleted from messages field?
-stop words, punctuation, emoji, 

lematization or stemming
output of wrangling - corpus can be part of a data frame with other cleaned up columns

- Why removing meta-data columns? There has to be a data-science driven reason ...
    1) can't delete columns from dataset without approval from owner of data
    2) just because I don't see a connection doesn't mean there is one
    3) how can use the information that we're given to create a potential feature that is a measure the level of intention - like or don't like

- Null values implies values that are missing. Is it possible to have values that are missing that are not null?
    1) cases where values are not null but can still be not good
 
- How would we check for data quality?
    1) max, min, range, content, etc.

- There seem to be many columns with about 80% of values that are missing. How do we deal with this?
    1) Could limit the data to the ~20% of the data with identifiers
    2) concentrate on comment_text and response variable initially?
    
    a) would it make sense to compute them myself?

- Do we know what columns are categorical and what columns are non-categorical? 
    1) I would say the identity labels are categorical
    2) target is the response
    3) what would comment_text be?
    non-categorical predominantly - continuous metric
    categorical - comment_text, refers to a class

- Think about ways of persisting the dataset as an output of the wrangling phase, which will be an input to the next phase storytelling). (Hint ... you can use something like Pickle, or perhaps generate a CSV file)
    1) pickle the data

emoji - encoded to a string as a token (possibility); do search on how to deal with emoji in NLP

packages - nltk, spacy

check if summation of identity labels across rows is equal to 1.0

# NLP Things

unicode remove, freq count is an option, every emoji maps to a unicode character and maybe can map it to a text entity. 


frequency stopwords, manual inspection
think about & determine - punc, stop words, conjunctions

In [ ]:
# print(string.punctuation)


table = str.maketrans('', '', string.punctuation)
print(table)

In [ ]:
import string

# split into words by white space
subset_train_data.loc[:,'comment_text_split'] = subset_train_data.loc[:,'comment_text'].apply(lambda x: x.split())
# remove punctuation from each word

table = str.maketrans('', '', string.punctuation)


In [ ]:
subset_train_data.comment_text_split.head()

In [ ]:
# create function to strip punctuation
import string
from string import punctuation

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)


In [ ]:
subset_train_data.loc[:,'comment_text_punct'] = subset_train_data.loc[:,'comment_text'].apply(lambda x: strip_punctuation(x))

In [ ]:
subset_train_data.comment_text_punct.head()

In [ ]:
## Count non-alphanumerical characters in sample
count_special = 0
for line in subset_train_data_v2['comment_text']:
    count_special += sum(not x.isalnum() for x in line)
print(count_special)

In [ ]:
# Quantify '!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?" in sample data
count = 0
for line in subset_train_data_v2['comment_text']:
    count += sum(1 for i in line if i in ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?"))
          
print("Total number of punctuation characters exists in string: ")
print(count);  

In [ ]:
# Attemp to check for accented characters

def count_ascii(text):
    if text.isascii() == False:
        return 1
    else:
        return 0

subset_train_data.loc[:,'comment_text_ascii'] = subset_train_data.loc[:,'comment_text'].apply(lambda x: count_accented_chars(x))

In [ ]:
print('Accented characters percentage in sample data: ', round(subset_train_data['comment_text_ascii'].sum() / subset_train_data['comment_text_ascii'].count() * 100,4), '%')

In [ ]:
# create comment field in all lower case

train_data['comment_lower'] = train_data['comment_text'].str.lower()

In [ ]:
train_data['comment_lower'].head()

Instead of removing emojis or emoticons, we'll convert them into words.

In [ ]:
# function to convert emoji and emoticons to words

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text
# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text


In [ ]:
# create new comment field with converted emoji and emoticon

train_data['comment_transform'] = train_data['comment_lower'].apply(convert_emoticons)
train_data['comment_transform'] = train_data['comment_lower'].apply(convert_emojis)

In [ ]:
train_data['comment_transform'].max()

In [ ]:
# Remove accented characters - convert to standardized ASCII characters

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

train_data.loc[:,'comment_text_cleaned'] = train_data.loc[:,'comment_text'].apply(lambda x: remove_accented_chars(x))

In [ ]:
# removing contractions 

def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

train_data.loc[:,'comment_text_cleaned'] = train_data.loc[:,'comment_text_cleaned'].apply(lambda x: expand_contractions(x))

based on results of the model, may need to go back and do more cleaning.

order of complexity
-stopwords and punctuation with NLTK
    - be careful with apostrophe. ex - didn't -> didnt as a token (wont need to worry about contractions)
    - if by taking care of punctuation, could take care of contractions (don't split)
    - think about by-product
    
- accented words
    - do we have a ratio or amount? is it worth while to do? 
    
- extract count of emoji/emoticons from text
    what proportion is it?
    
-lemmitization or stemming, one not both

-spelling
    -is it quantifiable; compare against a dictionary? not important for the sake of spelling but for consistency across the corpus
    
- split into a more manageable size to do my experimentation then apply to full data (random sample)

In [ ]:
# Create pickle file for use in the future

train_data.to_pickle(dst + '/cleaned_train_data.pkl')